# Last FM hometask <br>
https://www.kaggle.com/ravichaubey1506/lastfm <br>
1. Выбрать данные по странам своей группы (совместно): <br>
    3530203_70101: Germany, Netherlands <br>
    3530203_70102: Belarus, Ukraine, Poland, Russian Federation<br>
    3530903_70301: Sweden, Finland, Norway, Denmark, Iceland<br>
    3530903_70302: Spain, Portugal, France, Italy, Belgium<br>
    
2. Попытаться найти полезные с точки зрения продвижения групп (или еще чего-нибудь) и нетривиальные правила, используя алгоритмы Apriori, FPGrowth, FPMax и всевозможные метрики. Хотя бы 5 правил.
3. Вывести эти правила в отдельных ячейках. 
4. Подумать, как можно было бы использовать полученные правила.

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv("lastfm.csv")
our_countries = ['Sweden','Finland','Norway','Denmark','Iceland']
data.head()

,user,artist,sex,country
0,1,red hot chili peppers,f,Germany
1,1,the black dahlia murder,f,Germany
2,1,goldfrapp,f,Germany
3,1,dropkick murphys,f,Germany
4,1,le tigre,f,Germany


In [3]:
data = data[data.country.isin(our_countries)]
data.head()

data['country'].value_counts()

Sweden     12379
Finland    10157
Norway      4641
Denmark     2683
Iceland      408
Name: country, dtype: int64

In [4]:
new_data = data.groupby(['user', 'sex'])['artist'].apply(lambda x: ','.join(x)).reset_index()
new_data

,user,sex,artist
0,5,m,"dream theater,ac/dc,metallica,iron maiden,bob ..."
1,7,m,"soundgarden,stone temple pilots,buckethead,dre..."
2,19,f,"the kills,a perfect circle,crystal castles,bec..."
3,28,m,"toto,the doors,kent,the smiths,pink floyd,sex ..."
4,48,m,"as i lay dying,skillet,atreyu,bullet for my va..."
...,...,...,...
1429,19582,m,"the doors,gorillaz,ayreon,queens of the stone ..."
1430,19586,m,"bad religion,millencolin,the hives,foo fighter..."
1431,19587,m,"depeche mode,n*e*r*d,radiohead,snow patrol,the..."
1432,19605,f,"jens lekman,kings of convenience,the smiths,of..."


In [5]:
new_data = pd.concat([new_data.drop(['sex','artist'], axis=1), new_data['artist'].str.get_dummies(',')],
                      axis = 1).set_index(['user'])
new_data

,...and you will know us by the trail of dead,2pac,3 doors down,30 seconds to mars,311,36 crazyfists,44,50 cent,65daysofstatic,[unknown],...,wilco,within temptation,wolfgang amadeus mozart,wu-tang clan,yann tiersen,yeah yeah yeahs,yellowcard,yo la tengo,zero 7,Édith piaf
user,,,,,,,,,,,,,,,,,,,,,
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
28,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
48,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19582,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19586,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19587,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
from mlxtend.frequent_patterns import apriori, association_rules, fpgrowth, fpmax

In [7]:
frequent_set_apriori = apriori(new_data, min_support=0.02, use_colnames=True)
frequent_set_fpgrowth = fpgrowth(new_data, min_support=0.01, use_colnames=True)
frequent_set_fpmax = fpmax(new_data, min_support=0.04, use_colnames=True)

In [8]:
rules_confidence = association_rules(frequent_set_apriori, metric="confidence", min_threshold=0.25)
rules_confidence = rules_confidence.sort_values(by='support', ascending=False, na_position='last')
rules_confidence.head(7)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
138,(metallica),(in flames),0.164575,0.154812,0.057183,0.347458,2.244388,0.031705,1.295224
137,(in flames),(metallica),0.154812,0.164575,0.057183,0.369369,2.244388,0.031705,1.324746
150,(in flames),(system of a down),0.154812,0.137378,0.049512,0.319820,2.328029,0.028244,1.268226
72,(muse),(coldplay),0.134589,0.172245,0.049512,0.367876,2.135764,0.026330,1.309481
71,(coldplay),(muse),0.172245,0.134589,0.049512,0.287449,2.135764,0.026330,1.214526
149,(system of a down),(in flames),0.137378,0.154812,0.049512,0.360406,2.328029,0.028244,1.321445
203,(metallica),(system of a down),0.164575,0.137378,0.048815,0.296610,2.159081,0.026206,1.226378


In [9]:
rules_confidence[(rules_confidence['antecedents']=={'in flames'})]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
137,(in flames),(metallica),0.154812,0.164575,0.057183,0.369369,2.244388,0.031705,1.324746
150,(in flames),(system of a down),0.154812,0.137378,0.049512,0.319820,2.328029,0.028244,1.268226
140,(in flames),(nightwish),0.154812,0.133194,0.046722,0.301802,2.265884,0.026102,1.241490
44,(in flames),(children of bodom),0.154812,0.078103,0.039052,0.252252,3.229730,0.026960,1.232898
147,(in flames),(soilwork),0.154812,0.060669,0.039052,0.252252,4.157813,0.029659,1.256213


> Если ты слушаешь in flames, то тебе могут понравится следующие исполнители: 
    metallica, system of a down, nightwish, children of bodom, soilwork

In [10]:
rules_confidence = association_rules(frequent_set_fpgrowth, metric="confidence", min_threshold=0.25)
rules_confidence = rules_confidence[(rules_confidence['confidence'] < 0.8)]
rules_confidence = rules_confidence.sort_values(by='lift', ascending=False, na_position='last')
rules_confidence.head(7)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
1199,"(jay-z, nas)","(kanye west, snoop dogg)",0.016736,0.016736,0.011158,0.666667,39.833333,0.010877,2.949791
1204,"(kanye west, snoop dogg)","(jay-z, nas)",0.016736,0.016736,0.011158,0.666667,39.833333,0.010877,2.949791
1201,"(jay-z, snoop dogg)","(nas, kanye west)",0.018131,0.020223,0.011158,0.615385,30.429708,0.010791,2.547420
1202,"(nas, kanye west)","(jay-z, snoop dogg)",0.020223,0.018131,0.011158,0.551724,30.429708,0.010791,2.190323
1200,"(jay-z, kanye west)","(nas, snoop dogg)",0.025105,0.016039,0.011158,0.444444,27.710145,0.010755,1.771130
1203,"(nas, snoop dogg)","(jay-z, kanye west)",0.016039,0.025105,0.011158,0.695652,27.710145,0.010755,3.203228
2170,(the game),"(notorious b.i.g., kanye west)",0.023710,0.016039,0.010460,0.441176,27.506394,0.010080,1.760772


> Можно сделать вывод, что все эти исполнители схожи, поэтому их можно рекомендовать взаимозаменяемо.

In [11]:
data_with_country = data.groupby(['user', 'country'])['artist'].apply(lambda x: ','.join(x)).reset_index()
data_with_country = pd.concat([data_with_country.drop(['country','artist'], axis=1), 
                             pd.get_dummies(data_with_country['country']),
                             data_with_country['artist'].str.get_dummies(',')
                            ], axis = 1).set_index(['user'])

In [12]:
sets_with_country_apriori = apriori(data_with_country, min_support=0.03, use_colnames=True)

In [23]:
rules_with_country = association_rules(sets_with_country_apriori, min_threshold=0.02, metric="confidence")
rules_with_country = rules_with_country[(rules_with_country['antecedents'] == {'ac/dc'})]
rules_with_country.head(5).sort_values(by='lift', ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
140,(ac/dc),(metallica),0.087866,0.164575,0.040446,0.460317,2.797014,0.025986,1.547994
0,(ac/dc),(Finland),0.087866,0.355649,0.036960,0.420635,1.182726,0.005710,1.112168
60,(ac/dc),(Sweden),0.087866,0.400976,0.032078,0.365079,0.910476,-0.003154,0.943462


> ac/dc обязаны приехать в Финляндию

> ac/dc не слушают в Швеции, им там лучше не появляться